In [1]:
import requests
import pandas as pd

# Find governance-related parameters in the [Gnosis Safe contract](https://github.com/gnosis/safe-contracts/blob/main/contracts/GnosisSafe.sol)
From the contract text, identify parameters and their contexts (description and function for which each is defined), then find parameters which may be related to governance by defining keywords and searching for these within the parameter descriptions.

In [2]:
# Get content of Gnosis Safe contract
url = 'https://raw.githubusercontent.com/gnosis/safe-contracts/main/contracts/GnosisSafe.sol'
content = requests.get(url).text

In [3]:
# Assuming every relevant function has the @dev decorator description,
# split content into blocks of text containing one well-documented function
# (these may also contain other functions, which are ignored)
functions = content.split('@dev')

df_params = pd.DataFrame()
df_funcs = pd.DataFrame()

for f in functions[1:]:
    lines = [s.strip() for s in f.splitlines()]
    
    # Find (first line of) description of the function
    funcDescription = lines[0] 
    funcs = [(i, s.split(' ')[1]) for (i, s) in enumerate(lines) if s.startswith('function')]
    if len(funcs) > 0:
        funcName = funcs[0][1].split('(')[0]
        funcIndex = funcs[0][0]
    else:
        funcName = 'N/A'
        funcIndex = -1
    
    # Find the parameters and their descriptions
    paramLines = [s.split('@param')[-1].strip().split(' ', 1) for s in lines[:funcIndex] if '@param' in s]
    paramDicts = [{'function': funcName,
                   'parameter': p[0],
                   'description': p[1]}
                  for p in paramLines]
    
    # Add function and its parameters to DataFrames
    df_funcs = df_funcs.append({'name': funcName, 'description': funcDescription}, ignore_index=True)
    df_params = df_params.append(pd.DataFrame(paramDicts), ignore_index=True)

In [4]:
print("Functions:")
display(df_funcs)
print("Parameters:")
display(df_params)

Functions:


,name,description
0,setup,Setup function sets initial storage of contract.
1,execTransaction,Allows to execute a Safe transaction confirmed...
2,checkSignatures,Checks whether the signature provided is valid...
3,checkNSignatures,Checks whether the signature provided is valid...
4,requiredTxGas,Allows to estimate a Safe transaction.
5,approveHash,Marks a hash as approved. This can be used to ...
6,getChainId,Returns the chain id used by this contract.
7,encodeTransactionData,Returns the bytes that are hashed to be signed...
8,getTransactionHash,Returns hash to be signed by owners.


Parameters:


,function,parameter,description
0,setup,_owners,List of Safe owners.
1,setup,_threshold,Number of required confirmations for a Safe tr...
2,setup,to,Contract address for optional delegate call.
3,setup,data,Data payload for optional delegate call.
4,setup,fallbackHandler,Handler for fallback calls to this contract
5,setup,paymentToken,Token that should be used for the payment (0 i...
6,setup,payment,Value that should be paid
7,setup,paymentReceiver,Address that should receive the payment (or 0 ...
8,execTransaction,to,Destination address of Safe transaction.
9,execTransaction,value,Ether value of Safe transaction.


In [5]:
# Define keywords to search for within parameter descriptions.
# Currently this list is just based on my current understanding of what 
# "governance" consists of in this context...
# However, this list could be pulled from other documentation on Gnosis Safe
# and/or from general governance structure terminology on Govbase.
KEYWORDS = ['required', 'owner', 'approve']

def find_keywords(s):
    """Return list of keywords contained within string s, if any; otherwise, return False"""
    s = s.lower()
    keywordsFound = [k for k in KEYWORDS if k in s]
    if len(keywordsFound) == 0:
        keywordsFound = False
    return keywordsFound

# Filter parameters by keywords
df_params['contains_keywords'] = df_params['description'].apply(find_keywords)
df_params_gov = df_params[df_params['contains_keywords'] != False]

In [6]:
print("Functions containing governance-related keywords:")
display(df_params_gov)

Functions containing governance-related keywords:


,function,parameter,description,contains_keywords
0,setup,_owners,List of Safe owners.,[owner]
1,setup,_threshold,Number of required confirmations for a Safe tr...,[required]
20,checkSignatures,signatures,Signature data that should be verified. Can be...,[approve]
23,checkNSignatures,signatures,Signature data that should be verified. Can be...,[approve]
24,checkNSignatures,requiredSignatures,Amount of required valid signatures.,[required]
29,approveHash,hashToApprove,The hash that should be marked as approved for...,[approve]
